<a href="https://colab.research.google.com/github/szkjiro/program/blob/main/PyRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 回帰分析

ここではデータ解析環境 R で提供されている標準データセット cars を利用して，
散布図および回帰直線の描画をまず振り返り，
次に同じデータセットに対して回帰分析を用いた予測を扱っていく．

## データセットの読み込みと回帰直線の描画

Pythonでは statsmodels に R　の標準データセットが用意されている．
最初に，それを変数 cardata に読み込み，そのデータセットの先頭部分を関数 head により確認する．
変数に対して角かっこ［］で参照する場合とそうでない場合の違いなどは説明していない．
変数 speed はマイル単位であり，距離はフィート単位である．

In [ ]:
import statsmodels.api as sm
cardata = sm.datasets.get_rdataset('cars', 'datasets').data
# speed as miles, dist as feet
cardata.head()

ここでは，あえて変換しないが，speedの数値を1.6で割ればキロメートル単位になるし，distの値を0.3で割ればメートル単位になる．

データセットcardataの情報（5数要約）を表示しておく．

In [ ]:
cardata.describe()

データセットcardataを散布図に描画する．「・dist」の部分は凡例（legend）である．plot関数にオプションを指定したので，「・」で描画し，横軸に軸名を表示している．

In [ ]:
cardata.plot(x='speed', style='o')

次の例では散布図とともに回帰直線を表示している．データセットcardataの読み込み設定から繰り返しておいた．
散布図と回帰直線を同時に表示する関数を，
ライブラリseabornで提供している．
関数regplotは回帰直線とともに，その95%信頼区間を塗りで示している．

In [ ]:
import statsmodels.api as sm
import seaborn as sns

cardata = sm.datasets.get_rdataset('cars', 'datasets').data
cars = sns.regplot(x='speed', y='dist', data=cardata)


## 回帰分析

いよいよ回帰分析である．
ここから何段階か後に，予測直線を描画するが，図的には上の回帰直線と同じである．
図が同じだから同じものであるということはない．
直線の利用目的が違うのである．
予測直線は人工知能を用いたビッグデータ分析の中でも，最も多用されている一つである．

ライブラリ scikit-learn（プログラム内では sklearn で参照）は機械学習ライブラリの中でも，最も広く用いられている．
このライブラリにある LinearRegression 関数が回帰分析を行う．
関数 fit はデータセットに当てはまる直線を求め，predictはその直線に沿った予測値を求める．

次のプログラムでは回帰直線を描かせるプログラムのために，変数Kaikiを準備する．

In [ ]:
# Pythonでは a,b,c = d,e,f のような代入文により，
# 一度に複数の変数に対応順に代入することができる．
X, y = cardata[['speed']], cardata[['dist']]

from sklearn.linear_model import LinearRegression
Kaiki = LinearRegression()
Kaiki.fit(X, y)

次のプログラムで回帰直線を図示する．


In [ ]:
import numpy as np
import pandas as pd

Speed = pd.DataFrame({'speed': np.linspace(min(cardata.speed),
                                            max(cardata.speed),
                                            100)})
Speed['linear model'] = Kaiki.predict(Speed)
pd.concat([cardata, Speed]).plot(x='speed', style=['o', '-'])

予測値（関数predict）の結果を示すため，元のデータセット（データフレーム）にydashの名前で予測値を追加した．
予測値と元データとの差を残差といい，回帰直線（予測直線）の良さを測る上で重要な統計量である．
入門としの本授業ではその統計解析までは進めない．

次のプログラムではデータセットcardataの再定義から始めている．

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

cardata = sm.datasets.get_rdataset('cars', 'datasets').data
X, y = cardata[['speed']], cardata[['dist']]

Kaiki = LinearRegression()
Kaiki.fit(X, y)
# ここまではすでに実行したものと同じ

ydash = Kaiki.predict(X)
cardata['ydash'] = ydash
pd.options.display.float_format = ('{:.2f}'.format)
cardata['zansa'] = y - ydash
cardata.head()

予測値とは予測直線上の値である．実際のデータセットの各点に対して，予測値とデータとの間を点線でつないでみた．
この点線の長さを残差という．


In [ ]:
carplotset = cardata.plot(x='speed', y='dist', style='o', legend=False)
cardata.plot(x='speed', y='ydash', style='-', legend=False, ax=carplotset)
carplotset.vlines(x=X, ymin=y, ymax=ydash, linestyles='dotted')

統計学ではよく利用することだが，このような「誤差」に相当する量の分析には，平方和の平均値を扱う．
次の値がそれであり，残差平方和の平均である．
この値の解釈までは，入門である本授業では説明しない．

In [ ]:
mean_squared_error(y, ydash)**0.5